In [1]:
import IPython

In [2]:
from yargy.interpretation import fact as yrg_fact, attribute as yrg_attr
from yargy.pipelines import morph_pipeline as yrg_morph_pipeline
from yargy import rule as yrg_rule, or_ as yrg_r_or
from yargy.predicates import eq as yrg_rp_eq
from yargy import Parser as YrgParser

from ipymarkup import show_span_ascii_markup as natasha_show_markup

In [3]:
def show_matches(rule, *lines):
    parser = YrgParser(rule)
    for line in lines:
        matches = parser.findall(line)
        matches = sorted(matches, key=lambda _: _.span)
        spans = [_.span for _ in matches]
        natasha_show_markup(line, spans)
        if matches:
            facts = [_.fact for _ in matches]
            if len(facts) == 1:
                facts = facts[0]
            IPython.display.display(facts)

In [4]:
YRG_RP_DOT = yrg_rp_eq('.')

In [5]:
o_coat = yrg_fact(
    'Coat',
    ['type'],
)

In [6]:
a_demiseason = yrg_morph_pipeline([
    'демсезон',
    'демисезонный',
]).interpretation(
    o_coat.type.const('демисезонное')
)

a_man = yrg_morph_pipeline([
    'мужской',
    'муж',
]).interpretation(
    o_coat.type.const('мужское')
)

a_woman = yrg_morph_pipeline([
    'женский',
    'жен',
]).interpretation(
    o_coat.type.const('женское')
)

In [7]:
n_coat = yrg_morph_pipeline([
    'пальто',
    'полупальто',
])

r_coat = yrg_rule(
    yrg_r_or(a_man, a_woman).optional(),
    YRG_RP_DOT.optional(),
    a_demiseason.optional(),
    n_coat,
    YRG_RP_DOT.optional(),
).interpretation(
    o_coat
)

In [8]:
show_matches(
    r_coat,
    'пальто',
    'несколько пальто',
    'мужское полупальто',
    'жен. демисезонное пальто',  # finds only demiseason and skips a_woman
    'мужской пальто',
)

пальто
──────


.venv/lib/python3.12/site-packages/pymorphy2/analyzer.py:114: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Coat(
    type=None
)

несколько пальто
          ──────


Coat(
    type=None
)

...

Coat(
    type='мужское'
)

жен. демисезонное пальто
────────────────────────


Coat(
    type='демисезонное'
)

мужской пальто
──────────────


Coat(
    type='мужское'
)